# Data Processing and Metadata Enrichment Pipeline

This notebook guides you through a step-by-step process for loading, processing, and analyzing a dataset using a combination of custom scripts. The workflow includes loading data, creating metadata, filtering data, and performing fuzzy matching.

## Steps:
1. Set up environment and import necessary modules.
2. Define and check dataset directory.
3. Load and cache dataframes.
4. Create and display metadata.
5. Fetch, compare, and configure data fields.
6. Filter and process dataframes.
7. Perform fuzzy matching.
8. Save processed data.

## Step 1: Setup Environment

We begin by importing the necessary libraries and functions.


In [1]:

# Import necessary libraries and custom modules
import os
import json
import pandas as pd
import gc
import numpy as np



# Now, import the necessary custom functions from the scripts

from src.scripts.df_metadata import display_metadata_dfs, create_metadata_dfs, enrich_metadata_df
from src.scripts.fetch_data_fields import fetch_and_compare_data_fields
from src.scripts.build_data_fields_config import build_data_fields_config


# Change the current working directory to 'src'
os.chdir(os.path.join(os.getcwd(), 'src'))

## Step 2: Define and Check Dataset Directory

Define the dataset directory and ensure it exists. This step is crucial as it sets the working directory for subsequent operations.


In [2]:
from src.scripts.df_generator import check_directory_exists, load_or_cache_dataframes, show_loaded_dfs
# Define the dataset directory
notebook_directory =os.getcwd()  # This points to the root where the notebook is
dataset_directory = os.path.join(notebook_directory, 'dataset')

# Check if the dataset directory exists
if not check_directory_exists(dataset_directory):
    print(f"Error: Directory dataset does not exist.")
else:
    print(f"Dataset directory found")


Dataset directory found


## Step 3: Load and Cache DataFrames

Load the data from the dataset directory into pandas DataFrames. The data can be loaded from cache or directly from the source files if the cache is not available.


In [3]:
# Directory to store cached DataFrames
CACHE_DIR = os.path.join(notebook_directory, 'data', 'cache') 

# Optionally, you can define a list of specific files to process
specific_files = ['fr.openfoodfacts.org.products.csv']  # Set to None to process all files

# Load DataFrames from cache or source files
dfs = load_or_cache_dataframes(dataset_directory, CACHE_DIR, file_list=specific_files, separator='\t')

# Check if DataFrames are loaded
if not dfs:
    print("No DataFrames were loaded. Exiting.")
else:
    print(f"Loaded DataFrames: {list(dfs.keys())}")
    show_loaded_dfs(dfs, df_names=None)


Loaded 'fr.openfoodfacts.org.products' from cache.
Nullity matrix for 'fr.openfoodfacts.org.products' has been generated.
Loaded DataFrames: ['fr.openfoodfacts.org.products']
Currently loaded DataFrames:
DataFrame for file 'fr.openfoodfacts.org.products (320767, 146)':
            code                                                url  ... nutrition-score-fr_100g nutrition-score-uk_100g
0  0000000003087  http://world-fr.openfoodfacts.org/produit/0000...  ...                     NaN                     NaN
1  0000000004530  http://world-fr.openfoodfacts.org/produit/0000...  ...                    14.0                    14.0
2  0000000004559  http://world-fr.openfoodfacts.org/produit/0000...  ...                     0.0                     0.0
3  0000000016087  http://world-fr.openfoodfacts.org/produit/0000...  ...                    12.0                    12.0
4  0000000016094  http://world-fr.openfoodfacts.org/produit/0000...  ...                     NaN                     NaN

[5 

<Figure size 1500x900 with 0 Axes>

## Step 4: Create and Display Metadata

Generate metadata for the loaded DataFrames and display it to understand the structure and content of the data.


In [4]:
# Create metadata DataFrames
metadata_dfs = create_metadata_dfs(dfs)

# Check if metadata DataFrames were created
if not metadata_dfs:
    print("No metadata DataFrames were created. Exiting.")
else:
    print(f"Created Metadata DataFrames: {list(metadata_dfs.keys())}")
    display_metadata_dfs(metadata_dfs)


Created Metadata DataFrames: ['fr.openfoodfacts.org.products']
Metadata for fr.openfoodfacts.org.products (146, 8):
                          Column Name    Dtype  ... Duplicate Percentage  Missing Percentage
0                                code   object  ...             0.000000            0.007170
1                                 url   object  ...             0.000000            0.007170
2                             creator   object  ...            98.897947            0.000624
3                           created_t   object  ...            40.901410            0.000935
4                    created_datetime   object  ...            40.899993            0.002806
..                                ...      ...  ...                  ...                 ...
141  collagen-meat-protein-ratio_100g  float64  ...            96.363636           99.948561
142                        cocoa_100g  float64  ...            91.139241           99.704458
143             carbon-footprint_100g  float64 

## Step 5: Fetch, Compare, and Configure Data Fields

Fetch and compare data fields from the dataset, and build the necessary configuration files.


In [5]:
DATA_DIR = os.path.join(notebook_directory,'data')

# Run the fetch and compare data fields script
fetch_and_compare_data_fields(DATA_DIR)

# Build the config file
build_data_fields_config()


Created HISTORY_DIR: True
Created DIFF_DIR: True
No changes detected on data\data_fields.txt.
Config file 'data_fields_config.json' has been updated and saved.


## Step 6: Filter and Process DataFrames

Filter the metadata and corresponding DataFrames, and save the filtered data.


In [6]:
# Load the config.json
script_dir = os.path.join(notebook_directory,'scritps')
config_path = os.path.join(notebook_directory, 'config', 'data_fields_config.json')

with open(config_path, 'r') as file:
    config = json.load(file)

# Enrich the metadata DataFrame
combined_metadata = pd.concat(metadata_dfs.values(), keys=metadata_dfs.keys()).reset_index(level=0).rename(columns={'level_0': 'DataFrame'})
combined_metadata = enrich_metadata_df(combined_metadata, config)


# Save the combined metadata DataFrame to a CSV file
output_dir = os.path.join(notebook_directory, 'data')
os.makedirs(output_dir, exist_ok=True)

combined_metadata_path = os.path.join(output_dir, 'combined_metadata.csv')
combined_metadata.to_csv(combined_metadata_path, index=False)
print(f"Combined metadata {combined_metadata.shape} has been saved or updated.")


Combined metadata (146, 11) has been saved or updated.


## Step 7: Identify columns cluster

Checking cluster of columns based on Duplicate(%) and Fill(%)



In [7]:
from src.scripts.plot_metadata_clusters import run_dash_app

combined_metadata_cluster = combined_metadata.copy()

# Run the Dash app  
run_dash_app(combined_metadata_cluster)


## Step 8: Analyse, fuzzy and other stuff

blabla



In [8]:
from src.scripts.df_filtering import filter_metadata_and_dataframes, process_dataframe
from src.scripts.df_fuzzywuzzy import fuzzy_dataframe


# Specify your datetime checks as a list of tuples
datetime_checks = [
    # ('created_t', 'created_datetime'),
    # ('last_modified_t', 'last_modified_datetime')
]

# Specify your field frequency checks as a list of tuples
field_checks = [
    #(['countries', 'countries_tags', 'countries_fr'], 'countries'),
    #(['ingredients_from_palm_oil_n', 'ingredients_that_may_be_from_palm_oil_n'], 'ingredients_palm_oil'),
    (['nutrition_grade_fr', 'nutrition-score-fr_100g', 'nutrition-score-uk_100g'], 'nutrition'),
    #(['brands_tags', 'brands'], 'brands'),
    #(['additives_n', 'additives', 'additives_tags', 'additives_fr'], 'additives'),
    #(['states', 'states_tags', 'states_fr'], 'states'),
    (['pnns_groups_1', 'pnns_groups_2'], 'pnns_groups')
    
]

# Columns to check for at least one non-null value
columns_to_check = [
    'nutrition_grade_fr', 'energy_100g', 'fat_100g', 'saturated-fat_100g',
    'trans-fat_100g', 'cholesterol_100g', 'carbohydrates_100g', 'sugars_100g',
    'fiber_100g', 'proteins_100g', 'salt_100g', 'sodium_100g', 'vitamin-a_100g',
    'vitamin-c_100g', 'calcium_100g', 'iron_100g', 'nutrition-score-fr_100g',
    'nutrition-score-uk_100g'
]

# Fields to be deleted after anaylysis
fields_to_delete = [
    'url', 'created_t', 'created_datetime','last_modified_t', 'last_modified_datetime', 'states', 'states_tags', 'states_fr', 'countries', 'countries_tags', 'countries_fr',
    'brands_tags', 'brands', 'additives_n', 'additives', 'additives_tags', 'additives_fr',
    'creator','ingredients_from_palm_oil_n', 'ingredients_that_may_be_from_palm_oil_n',
    'quantity', 'serving_size', 'additives', 'ingredients_text','product_name',
    'categories','categories_tags','categories_fr','packaging','packaging_tags','image_url','image_small_url','main_category','main_category_fr'
    ]

# Now process your specific DataFrame
df_name = 'fr.openfoodfacts.org.products'

# Filter and process DataFrame in one step
if df_name in dfs:
    combined_metadata, filtered_dfs = filter_metadata_and_dataframes(combined_metadata, dfs, 20)
    process_dataframe(filtered_dfs[df_name], log_dir='logs', temp_dir='temp', datetime_checks=datetime_checks, field_checks=field_checks)
    fuzzy_dataframe(temp_dir='temp', config_dir='config', checks=field_checks, threshold=90)

    # Drop rows where all specified columns are null
    filtered_dfs[df_name].dropna(subset=columns_to_check, how='all', inplace=True)

    # Drop duplicates after filtering rows
    filtered_dfs[df_name].drop_duplicates(inplace=True)

    # Delete the specified columns from combined_metadata and update the related DataFrame
    combined_metadata = combined_metadata[~combined_metadata['Column Name'].isin(fields_to_delete)]
    filtered_dfs[df_name] = filtered_dfs[df_name][combined_metadata['Column Name']]

    # Save the processed DataFrame to the dataset directory
    dataset_path = os.path.join('dataset', f'processed_{df_name}.csv')
    filtered_dfs[df_name].to_csv(dataset_path, index=False)
    
    # Save the updated metadata to the data directory
    metadata_path = os.path.join('data', f'processed_metadata.csv')
    combined_metadata.to_csv(metadata_path, index=False)
    
    print(f"Processed DataFrame '{df_name}' and metadata have been saved.")
else:
    print(f"DataFrame '{df_name}' not found in the loaded DataFrames.")


Updated DataFrame 'fr.openfoodfacts.org.products' to retain only relevant columns.
Check the nutrition combination file for more details about fields frequency.
Check the pnns_groups combination file for more details about fields frequency.
[nutrition] Total combinations to process: 441
[nutrition] Finished processing all combinations.
Grouped results for nutrition have been generated
[pnns_groups] Total combinations to process: 44
[pnns_groups] Finished processing all combinations.
Grouped results for pnns_groups have been generated
Fuzzy matching and grouping completed for all checks.
Processed DataFrame 'fr.openfoodfacts.org.products' and metadata have been saved.


## Step 9: Nutrition score Clustering Dashboard

blabla



In [9]:
from src.scripts.plot_nutriscore import run_dash_or_show_iframe, safe_eval
import os
import pandas as pd


# Change the current working directory to 'src'
# os.chdir(os.path.join(os.getcwd(), 'src'))
# notebook_directory =os.getcwd()

# Assuming notebook_directory is already defined in your notebook
nutriscore_directory = os.path.join(notebook_directory, 'temp', 'nutrition_combination_log.csv')
nutriscore = pd.read_csv(nutriscore_directory)

run_dash_or_show_iframe(nutriscore)



Running in Jupyter Notebook


### Nutrient Maximum Limits Justification

| Nutrient                      | Maximum Limit      | Justification |
|-------------------------------|--------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Energy (energy_100g)**       | 950 kcal/100g      | The upper limit of 950 kcal per 100g accounts for extremely energy-dense foods like pure oils and concentrated products, while capturing potential data entry errors without excluding valid outliers.                                                                                                       |
| **Fat (fat_100g)**             | 95g/100g           | While pure fat can reach 100g/100g, lowering the limit slightly to 95g/100g flags potential rounding errors in data entry, as it's rare for foods to contain exactly 100g of fat.                                                                                                                             |
| **Saturated Fat (saturated-fat_100g)** | 55g/100g  | High-saturated fat products like butter can have up to 50-60% saturated fat. A limit of 55g/100g allows flexibility for processed fats while still flagging extreme cases.                                                                                                                                   |
| **Carbohydrates (carbohydrates_100g)** | 95g/100g | Carbohydrates can theoretically reach 100% of a food's weight, but setting the limit at 95g/100g helps to flag data entry errors while accommodating foods with high carbohydrate content.                                                                                                                  |
| **Sugars (sugars_100g)**       | 95g/100g           | Sugars, although able to reach 100g/100g, are rarely that high in practice. Setting the limit at 95g/100g captures realistic values while identifying potential overstatements.                                                                                                                               |
| **Sodium (sodium_100g)**       | 3g/100g            | While most foods don't exceed 2.3g/100g, certain salt-heavy products like salted meats or fish can reach higher sodium levels. A 3g/100g limit captures these outliers while maintaining realistic boundaries.                                                                                                 |
| **Salt (salt_100g)**           | 6g/100g            | With sodium reaching 3g/100g in some extreme cases, the corresponding salt content would be around 6g/100g, maintaining logical sodium-salt relationships for highly salted products.                                                                                                                         |
| **Trans Fat (trans-fat_100g)**  | 5g/100g            | Modern food regulations limit trans fats in many countries, making it rare for foods to exceed 5g/100g. This lower limit ensures compliance with current guidelines and excludes unrealistic trans fat levels.                                                                                                |
| **Cholesterol (cholesterol_100g)** | 500mg/100g     | High-cholesterol foods like organ meats are accommodated, but a higher limit of 500mg/100g better captures naturally high-cholesterol foods without excluding legitimate entries.                                                                                                                             |
| **Fiber (fiber_100g)**         | 50g/100g           | Fiber content can be high in foods like bran, but a limit of 50g/100g ensures that even fiber-dense products are realistically capped, filtering out unrealistic entries.                                                                                                                                     |
| **Proteins (proteins_100g)**   | 90g/100g           | High-protein products, especially supplements, can reach up to 90g/100g. This limit allows for protein-dense foods while filtering out implausible data entries.                                                                                                                                              |
| **Vitamin A (vitamin-a_100g)** | 30mg/100g          | Foods like liver can contain high levels of Vitamin A, but 30mg/100g is a more conservative upper limit to ensure that extreme, potentially toxic levels are flagged as data errors.                                                                                                                           |
| **Vitamin C (vitamin-c_100g)** | 50mg/100g          | While some fruits have high Vitamin C concentrations, a 50mg/100g limit is sufficient to capture natural sources while identifying improbable values.                                                                                                                                                         |
| **Calcium (calcium_100g)**     | 30mg/100g          | Although fortified foods may exceed natural calcium levels, 30mg/100g is a reasonable limit that captures high-calcium foods while excluding artificially inflated entries.                                                                                                                                     |
| **Iron (iron_100g)**           | 40mg/100g          | Iron-rich foods like red meat and fortified cereals are accommodated, but a 40mg/100g limit is more realistic for naturally occurring iron levels, preventing data entry errors.                                                                                                                               |

### Additional Justification:
- **Nutritional Guidelines**: Limits are based on standard nutritional data from sources such as USDA, EFSA, and general dietary recommendations.
- **Data Integrity**: These limits ensure data is free from common errors (e.g., mistyping, incorrect unit conversions), helping to maintain clean, reliable data for analysis.


In [10]:
from src.scripts.df_generator import check_directory_exists, load_or_cache_dataframes, show_loaded_dfs
from src.scripts.df_business_data_integrity import run_integrity_check
from src.scripts.df_pnns_group import standardize_pnns_groups
from src.scripts.df_nutriscore import check_and_standardize_nutrition_grades

# Import necessary libraries and custom modules
import os

# Define the dataset directory
notebook_directory = os.getcwd()  # This points to the root where the notebook is
dataset_directory = os.path.join(notebook_directory, 'dataset')
json_file_path = os.path.join(notebook_directory, 'config', 'pnns_groups_grouped_results.json')

# Check if the dataset directory exists
if not check_directory_exists(dataset_directory):
    print(f"Error: Directory dataset does not exist.")
else:
    print(f"Dataset directory found")

# Directory to store cached DataFrames
CACHE_DIR = os.path.join(notebook_directory, 'data', 'cache')

# Optionally, you can define a list of specific files to process
specific_files = ['processed_fr.openfoodfacts.org.products.csv']

# Load DataFrames from cache or source files
processed_dfs = load_or_cache_dataframes(dataset_directory, CACHE_DIR, file_list=specific_files, separator=',')

# Check if DataFrames are loaded
if not processed_dfs:
    print("No DataFrames were loaded. Exiting.")
else:
    print(f"Loaded DataFrames: {list(processed_dfs.keys())}")
    show_loaded_dfs(processed_dfs, df_names=None)

# Get the specific DataFrame for processing
df_name = 'processed_fr.openfoodfacts.org.products'
if df_name in processed_dfs:
    df = processed_dfs[df_name]  # Pass the DataFrame directly if not a dictionary
    run_integrity_check(df, log_dir='logs')
    standardize_pnns_groups(json_file_path) 
    check_and_standardize_nutrition_grades(df)
    
    # Save the processed DataFrame to the dataset directory
    dataset_path = os.path.join('dataset', f'standardized_{df_name}.csv')
    processed_dfs[df_name].to_csv(dataset_path, index=False)
    print(f"Processed DataFrame '{df_name}' has been updated.")
else:
    print(f"DataFrame '{df_name}' not found in the loaded DataFrames.")


Dataset directory found
Loaded 'processed_fr.openfoodfacts.org.products' from cache.
Nullity matrix for 'processed_fr.openfoodfacts.org.products' has been generated.
Loaded DataFrames: ['processed_fr.openfoodfacts.org.products']
Currently loaded DataFrames:
DataFrame for file 'processed_fr.openfoodfacts.org.products (262828, 21)':
            code nutrition_grade_fr  ... nutrition-score-fr_100g nutrition-score-uk_100g
0  0000000004530                  d  ...                    14.0                    14.0
1  0000000004559                  b  ...                     0.0                     0.0
2  0000000016087                  d  ...                    12.0                    12.0
3  0000000016094                NaN  ...                     NaN                     NaN
4  0000000016100                NaN  ...                     NaN                     NaN

[5 rows x 21 columns]


pnns_groups_1 and pnns_groups_2 combinations have been standardized.
Processed DataFrame 'processed_fr.openf

<Figure size 1500x900 with 0 Axes>

In [11]:
# Optionally, you can define a list of specific files to process
specific_files = ['standardized_processed_fr.openfoodfacts.org.products.csv']  # Set to None to process all files

# Load DataFrames from cache or source files
dfs_std = load_or_cache_dataframes(dataset_directory, CACHE_DIR, file_list=specific_files, separator=',')

# Check if DataFrames are loaded
if not dfs_std:
    print("No DataFrames were loaded. Exiting.")
else:
    print(f"Loaded DataFrames: {list(dfs_std.keys())}")
    show_loaded_dfs(dfs_std, df_names=None)


# Check if metadata DataFrames were created
if not dfs_std:
    print("No metadata DataFrames were created. Exiting.")
else:
    print(f"Created Metadata DataFrames: {list(dfs_std.keys())}")
    display_metadata_dfs(dfs_std)



Loaded 'standardized_processed_fr.openfoodfacts.org.products' from cache.
Nullity matrix for 'standardized_processed_fr.openfoodfacts.org.products' has been generated.
Loaded DataFrames: ['standardized_processed_fr.openfoodfacts.org.products']
Currently loaded DataFrames:
DataFrame for file 'standardized_processed_fr.openfoodfacts.org.products (262828, 21)':
            code nutrition_grade_fr  ... nutrition-score-fr_100g nutrition-score-uk_100g
0  0000000004530                  d  ...                    14.0                    14.0
1  0000000004559                  b  ...                     0.0                     0.0
2  0000000016087                  d  ...                    12.0                    12.0
3  0000000016094                NaN  ...                     NaN                     NaN
4  0000000016100                NaN  ...                     NaN                     NaN

[5 rows x 21 columns]


Created Metadata DataFrames: ['standardized_processed_fr.openfoodfacts.org.produ

<Figure size 1500x900 with 0 Axes>

In [12]:
# Import the run_dash_app function from the module
from src.scripts.plot_imputation import run_dash_app

# Pass your dataframe to the Dash app
#run_dash_app(dfs_std['standardized_processed_fr.openfoodfacts.org.products'])
#dfs_std['standardized_processed_fr.openfoodfacts.org.products'].dtypes

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import seaborn as sns
import matplotlib.pyplot as plt

# Load your data
df = dfs_std['standardized_processed_fr.openfoodfacts.org.products']

# Handle missing values: Keep track of which values are missing
missing_mask_energy = df['energy_100g'].isna()
missing_mask_fat = df['fat_100g'].isna()

# Convert categorical variables to string (if not already)
df['pnns_groups_1'] = df['pnns_groups_1'].astype(str)
df['pnns_groups_2'] = df['pnns_groups_2'].astype(str)

# Encode categorical variables using OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop='first')

# Train separate models for pnns_groups_1 and pnns_groups_2
encoded_categories = encoder.fit_transform(df[['pnns_groups_1', 'pnns_groups_2']])
encoded_df = pd.DataFrame(encoded_categories, columns=encoder.get_feature_names_out())
df = pd.concat([df, encoded_df], axis=1)

# Drop original categorical columns to prevent mixing types
df.drop(columns=['pnns_groups_1', 'pnns_groups_2'], inplace=True)

# Define features
features = ['energy_100g', 'fat_100g', 'saturated-fat_100g', 'trans-fat_100g',
            'cholesterol_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g',
            'proteins_100g', 'salt_100g', 'sodium_100g', 'vitamin-a_100g',
            'vitamin-c_100g', 'calcium_100g', 'iron_100g']

# Step 1: Train models on non-missing data for each feature
predictions = {}

for feature in features:
    # Use only rows where the target feature is not NaN
    complete_data = df.dropna(subset=[feature])
    
    if complete_data.empty:  # Skip if there are no complete rows
        continue
    
    X = complete_data.drop(columns=[feature])
    y = complete_data[feature]
    
    # Ensure all features are numeric
    X = X.select_dtypes(include=['number'])  # Keep only numeric columns
    
    # Train the model
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X, y)
    
    # Store the model for later use
    predictions[feature] = model

# Step 2: Predict missing values using the trained models
def predict_missing_values(df, predictions):
    for feature, model in predictions.items():
        missing_mask = df[feature].isna()
        if missing_mask.any():
            X_missing = df[missing_mask].drop(columns=[feature])
            # Ensure all features are numeric
            X_missing = X_missing.select_dtypes(include=['number'])  # Keep only numeric columns
            # Predict missing values
            df.loc[missing_mask, feature] = model.predict(X_missing)
    return df

# Fill in missing values iteratively
df = predict_missing_values(df, predictions)

# Step 3: Predict pnns_groups_1 based on features
# Create a separate model for predicting pnns_groups_1
X_pnns = df.dropna(subset=['pnns_groups_1'])
y_pnns = X_pnns['pnns_groups_1']
X_pnns = X_pnns.drop(columns=['pnns_groups_1'])

# Ensure to encode categorical features if needed
X_pnns = encoder.transform(X_pnns[['pnns_groups_1', 'pnns_groups_2']])
X_pnns = pd.DataFrame(X_pnns, columns=encoder.get_feature_names_out())

# Train the classifier
pnns_model = RandomForestClassifier(n_estimators=100, random_state=42)
pnns_model.fit(X_pnns, y_pnns)

# Predict missing pnns_groups_1
missing_pnns_mask = df['pnns_groups_1'].isna()
if missing_pnns_mask.any():
    X_missing_pnns = df[missing_pnns_mask].drop(columns=['pnns_groups_1'])
    # Ensure to encode categorical features if needed
    X_missing_pnns_encoded = encoder.transform(X_missing_pnns[['pnns_groups_1', 'pnns_groups_2']])
    df.loc[missing_pnns_mask, 'pnns_groups_1'] = pnns_model.predict(X_missing_pnns_encoded)

# Visualize the relationship between features and pnns_groups
sns.scatterplot(data=df, x='energy_100g', y='fat_100g', hue='pnns_groups_1', alpha=0.7)
plt.title('Energy vs Fat colored by PNNS Groups')
plt.show()


ValueError: could not convert string to float: 'd'